In [1]:
import pandas as pd

import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive



In [2]:
# Load Data
article = pd.read_csv("/Users/aswathshakthi/PycharmProjects/MNP25/data/articles.csv")
district = pd.read_csv("/Users/aswathshakthi/PycharmProjects/MNP25/data/District_beneficiary.csv")
public = pd.read_excel("/Users/aswathshakthi/PycharmProjects/MNP25/data/Public_Beneficiary_21_24.xlsx")



article_name = article["Articles"].unique().tolist()

if article_name:
    for art in article_name:

        cpu = article[article["Articles"] == art]["Cost per unit"].tolist()[0]
        print(cpu)


100000.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
5000.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
20500.0
10000.0
20500.0
20500.0
20500.0
20500.0
5000.0


In [3]:
article

,Articles,Cost per unit,Item Type
0,Handicapped Hand Tricycle,100000.0,Article
1,Fixed Wheel Chair,20500.0,Article
2,Wheelchair,20500.0,Article
3,Handicapped Scooter,20500.0,Article
4,Two Wheeler,20500.0,Article
...,...,...,...
65,Sewing Machine Heavy,20500.0,Article
66,Sewing Machine ORD / Motor,20500.0,Article
67,Sewing Machine ORD,20500.0,Article
68,Gents Cycle,20500.0,Article


In [4]:
article

,Articles,Cost per unit,Item Type
0,Handicapped Hand Tricycle,100000.0,Article
1,Fixed Wheel Chair,20500.0,Article
2,Wheelchair,20500.0,Article
3,Handicapped Scooter,20500.0,Article
4,Two Wheeler,20500.0,Article
...,...,...,...
65,Sewing Machine Heavy,20500.0,Article
66,Sewing Machine ORD / Motor,20500.0,Article
67,Sewing Machine ORD,20500.0,Article
68,Gents Cycle,20500.0,Article


In [5]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import streamlit_authenticator as stauth
import yaml
import time
from yaml.loader import SafeLoader

master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"  # Replace with your file's ID
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


2025-02-01 22:35:15.166 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-01 22:35:15.170 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-01 22:35:15.172 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-01 22:35:15.175 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/Users/aswathshakthi/PycharmProjects/MNP25/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://g

In [6]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [7]:


def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [8]:
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"

public = read_file(public_data_id)
public_master = read_file(public_master_id)
public_master

Download progress: 100%
Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,Article Name,Cost Per Unit,Quantity,Total Value,Comments
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,NaN
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Educational Aid,15000,1,15000,Rs.15000 for children
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Educational Aid,15000,1,15000,Rs.15000
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,NaN
5,P 006,K.Malar,799405228901,Yes,"2/519 Selva Vinayakar Koil St, Thuraiakkam, Ch...",7401234296,Sewing Machine ORD / Motor,6300,1,6300,NaN
6,P 007,M.Viswanathan,281683221567,No,"NGO Nagar, Saravambakkam, Chenglepat",7358991411,Gents Cycle,5500,1,5500,NaN
7,P 008,Thenmozhi,902808983035,No,"312 North St, Villupuram",7695877623,Sewing Machine ORD / Motor,6300,1,6300,NaN
8,P 009,V.Sumathi,288406704750,No,"36,Mettu St, Vridhachalam, Cuddalore",9384920615,Sewing Machine ORD / Motor,6300,1,6300,NaN
9,P 010,Subhasree,364828263680,No,"18, Seethalakshmi Nagar, Pudhucherry",8667248576,Girls Cycle,5200,1,5200,NaN


In [17]:
import locale
pub_fund = 5000000 - public_master['Total Value'].sum()

print(f"<h5>Remaining Fund: ₹ <span style='color:{'Green' if pub_fund >= 0 else 'Red'};'>{pub_fund:,.0f}</span></h5>")

<h5>Remaining Fund: ₹ <span style='color:Green;'>4,412,334</span></h5>


In [13]:
district = read_file(master_data_id)
district

Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS
0,Andaman,Handicapped Aid,Aid,1,5000.0,5000.0,10000.0,NaN,Public
1,Andaman,Handicapped Aid,Aid,1,5000.0,5000.0,NaN,0.0,Private
2,Chengalpattu,Agri Power Sprayer (1.2 Bolo),Article,2,11760.0,23520.0,160000.0,NaN,NaN
3,Chengalpattu,Gents Cycle,Article,8,5500.0,44000.0,NaN,NaN,NaN
4,Chengalpattu,Girls Cycle,Article,8,5200.0,41600.0,NaN,NaN,NaN
5,Chengalpattu,Iron Box,Article,8,7000.0,56000.0,NaN,NaN,NaN
6,Chengalpattu,Sewing Machine ORD,Article,6,5200.0,31200.0,NaN,-36320.0,NaN
7,Krishnagiri-B3,Audio Mixer,Article,1,NaN,NaN,52250.0,52250.0,NaN
8,Salem,Girls Cycle,Article,1,5200.0,5200.0,600000.0,NaN,NaN
9,Salem,Iron Box,Article,2,7000.0,14000.0,NaN,NaN,NaN


In [15]:
saved_data = read_file(master_data_id)
existing_record = saved_data[
                    (saved_data["NAME OF THE DISTRICT"] == "Andaman") &
                    (saved_data["REQUESTED ARTICLE"] == "Handicapped Aid")]

Download progress: 100%


In [16]:
existing_record

,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS
0,Andaman,Handicapped Aid,Aid,1,5000.0,5000.0,10000.0,NaN,Public
1,Andaman,Handicapped Aid,Aid,1,5000.0,5000.0,NaN,0.0,Private


In [19]:
existing_record["QUANTITY"].values[0]

np.int64(1)

In [ ]:
st.header("Institution Applications")
inst_data_id = "1dOMubywUqJId2gXHwNWp185L3QmadUnwxyFf0DC9M1s"
inst_data = read_file(inst_data_id)
action = st.radio("Action", options=["Add", "Edit", "Delete"], horizontal=True)
# Input common fields
app_no = st.text_input("Application Number (e.g., I 001)")
if action == "Add":
    name = st.text_input("Name  of Institution")
    article_name = st.selectbox("Select Article Name", article["Articles"].unique().tolist())
    quantity = st.number_input("Quantity", min_value=1)
    address = st.text_area("Address")
    mobile = st.text_input("Mobile Number")
    comment = st.text_area("Comments")

    # Submit button for adding new records
    if st.button("Add"):
        try:
            # Validate application number and Aadhaar
            if not app_no:
                st.warning("Application Number cannot be empty.")
            elif app_no in inst_data["App. No."].values:
                st.error("Application Number already added to the Application.")

            else:
                # Create a new entry
                new_entry = {
                    "App. No.": app_no,
                    "Aadhar (Without Space)": str(aadhar),
                    "Name": name,
                    "Handicapped (Yes / No)": handicapped,
                    "Address": address,
                    "Mobile": mobile,
                    "Article Name": article_name,
                    "Comments": comment
                }

                # Append the new entry to the DataFrame
                public_master = pd.concat([public_master, pd.DataFrame([new_entry])],ignore_index=True
                ).sort_values(by=["App. No."], ascending=True).reset_index(drop=True)

                # Save the updated data back to the file
                update_file(public_master_id, public_master)

                # Success message and updated DataFrame display
                st.success(f"Application {app_no} added successfully!")
                st.data_editor(public_master)
        except Exception as e:
            st.error(f"An error occurred: {str(e)}")

    elif action == "Edit":

    if st.button("Search", key="search_button"):
        # Search for the record by Application Number
        record = public_master[public_master["App. No."] == app_no.strip()]

        if not record.empty:
            # Store record in session state for persistence
            st.session_state["edit_record"] = record.iloc[0].to_dict()
        else:
            st.error("Application not found.")

    if "edit_record" in st.session_state:

        # Retrieve the stored record from session state
        record = st.session_state["edit_record"]
        # Display fields for editing
        aadhar = st.text_input("Aadhaar Number", value=record["Aadhar (Without Space)"])
        name = st.text_input("Name", value=record["Name"])
        handicapped = st.radio(
            "Handicapped",
            options=["Yes", "No"],
            horizontal=True,
            index=["Yes", "No"].index(record["Handicapped (Yes / No)"]))
        address = st.text_area("Address", value=record["Address"])
        mobile = st.text_input("Mobile Number", value=record["Mobile"])
        article_name = st.selectbox("Select Article Name",
            article["Articles"].unique().tolist(),
            index=article["Articles"].tolist().index(record["Article Name"]))
        comment = st.text_area("Comments", value=record["Comments"])

        if st.button("Update", key="update_button"):
            try:
                # Update the record in the DataFrame
                public_master.loc[public_master["App. No."] == app_no, "Aadhar (Without Space)"] = aadhar
                public_master.loc[public_master["App. No."] == app_no, "Name"] = name
                public_master.loc[public_master["App. No."] == app_no, "Handicapped (Yes / No)"] = handicapped
                public_master.loc[public_master["App. No."] == app_no, "Address"] = address
                public_master.loc[public_master["App. No."] == app_no, "Mobile"] = mobile
                public_master.loc[public_master["App. No."] == app_no, "Article Name"] = article_name
                public_master.loc[public_master["App. No."] == app_no, "Comments"] = comment

                # Save the updated data back to the file
                update_file(public_master_id, public_master)
                st.success(f"Application {app_no} updated successfully!")
                st.session_state.pop("edit_record")  # Clear session state after update
                st.dataframe(public_master)
            except Exception as e:
                st.error(f"An error occurred: {str(e)}")

    elif action == "Delete":
    if st.button("Delete"):
        # Check if the record exists
        if app_no in public_master["App. No."].values:
            # Delete the record
            public_master = public_master[public_master["App. No."] != app_no]

            # Save the updated data back to the file
            update_file(public_master_id, public_master)

            st.success(f"Application {app_no} deleted successfully!")
        else:
            st.error("Application not found.")
    st.dataframe(public_master)

    st.download_button(
    label="Download Records",
    data=public_master.to_csv(index=False).encode('utf-8'),
    file_name="Public Beneficiaries Records.csv",
    mime="text/csv")
